In [1]:
# Import libraries
from GRANTA_MIScriptingToolkit import granta as mpy
import GRANTA_MIScriptingToolkit as gdl
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
import pandas as pd
#from wordcloud import WordCloud

In [2]:
mi = mpy.connect('http://azewacadmi1v1.win.ansys.com/mi_servicelayer/', autologon=True)
mi
mi.dbs

[<Database: ASM Medical Materials, unit system: Database>,
 <Database: BiCrystal Project, unit system: Database>,
 <Database: Sustainability, unit system: Database>,
 <Database: JAHM Curve Data, unit system: Database>,
 <Database: Lvl 3 - Built Environment, unit system: Database>,
 <Database: Restricted Substances, unit system: Database>,
 <Database: MI Asphalt Database, unit system: Database>,
 <Database: HighTemperatureSuperconductors_SynthesisCharacterization, unit system: Database>,
 <Database: Academic Resources Database, unit system: Database>,
 <Database: Additive Manufacturing Template, unit system: Database>,
 <Database: MI Product Design Projects, unit system: Database>,
 <Database: Main, unit system: Database>,
 <Database: MI_HC_Team, unit system: Database>,
 <Database: Composites Template, unit system: Database>,
 <Database: Human Biological Materials, unit system: Database>,
 <Database: MI Training, unit system: Database>,
 <Database: MI Blank Database, unit system: Databa

In [3]:
my_db = mi.get_db('Academic Resources Database')
my_db.tables

[<Table name: Granta Reference Papers, subsets: set()>,
 <Table name: Ansys Reference Papers, subsets: set()>,
 <Table name: Materials Sciences Literature, subsets: set()>,
 <Table name: Teaching, Research and Other Resources, subsets: set()>,
 <Table name: Authors / Professors / Researchers / Employees, subsets: set()>,
 <Table name: Student Teams, subsets: set()>,
 <Table name: Universities / Institutions / Laboratories / Companies, subsets: set()>,
 <Table name: Countries, subsets: set()>,
 <Table name: Societies and Communities, subsets: set()>,
 <Table name: Events, Conferences, Workshops, Symposia, subsets: set()>,
 <Table name: Collaboration, Sponsoring, (C)R&D Projects..., subsets: set()>,
 <Table name: Products Competitors, subsets: set()>]

In [4]:
my_db.tables[0].children

[<Record long name:Selector>,
 <Record long name:EduPack>,
 <Record long name:CES Software ?>,
 <Record long name:MI>,
 <Record long name:MDS>,
 <Record long name:_Reference Papers to Analyze_>]

In [5]:
session = gdl.GRANTA_MISession('http://azewacadmi1v1.win.ansys.com/mi_servicelayer/', autoLogon=True)
browseService = session.browseService
databases = browseService.GetDatabases().databases

print("Found {0} databases on the Granta MI Server".format(len(databases)))
for d in databases:
    print("Database key: {0.DBKey}, Database name: {0.name}".format(d))

Found 17 databases on the Granta MI Server
Database key: MI_ASM_Medical_Materials_1.54.3_2022_R1, Database name: ASM Medical Materials
Database key: MI_BiCrystalProject, Database name: BiCrystal Project
Database key: MU_Sandbox_Energy_systems, Database name: Sustainability
Database key: MI_JAHM_1.3.3m_2021_R2, Database name: JAHM Curve Data
Database key: Lvl3_BuildingEnvironment, Database name: Lvl 3 - Built Environment
Database key: MI_Restricted_Substances, Database name: Restricted Substances
Database key: MI_Asphalt, Database name: MI Asphalt Database
Database key: HighTemperatureSuperconductors_SynthesisCharacterization, Database name: HighTemperatureSuperconductors_SynthesisCharacterization
Database key: MI_RefPapers, Database name: Academic Resources Database
Database key: MI_Template_Additive_Manufacturing, Database name: Additive Manufacturing Template
Database key: MI_ProductDesign, Database name: MI Product Design Projects
Database key: EduPack, Database name: Main
Database 

In [6]:
df = pd.DataFrame({'DBKey': [db.DBKey for db in databases],
                  'DBName': [db.name for db in databases]})
df

,DBKey,DBName
0,MI_ASM_Medical_Materials_1.54.3_2022_R1,ASM Medical Materials
1,MI_BiCrystalProject,BiCrystal Project
2,MU_Sandbox_Energy_systems,Sustainability
3,MI_JAHM_1.3.3m_2021_R2,JAHM Curve Data
4,Lvl3_BuildingEnvironment,Lvl 3 - Built Environment
5,MI_Restricted_Substances,Restricted Substances
6,MI_Asphalt,MI Asphalt Database
7,HighTemperatureSuperconductors_SynthesisCharac...,HighTemperatureSuperconductors_SynthesisCharac...
8,MI_RefPapers,Academic Resources Database
9,MI_Template_Additive_Manufacturing,Additive Manufacturing Template


In [7]:
# Get Tables from Ref Papers DB
dbKey = "MI_RefPapers"
tables = browseService.GetTables(gdl.GetTables(DBKey=dbKey)).tableDetails

print("Found {0} tables in database {1}".format(len(tables), dbKey))
for t in tables:
    print("Table name: {0}".format(t.tableReference.name))

Found 12 tables in database MI_RefPapers
Table name: Granta Reference Papers
Table name: Ansys Reference Papers
Table name: Materials Sciences Literature
Table name: Teaching, Research and Other Resources
Table name: Authors / Professors / Researchers / Employees
Table name: Student Teams
Table name: Universities / Institutions / Laboratories / Companies
Table name: Countries
Table name: Societies and Communities
Table name: Events, Conferences, Workshops, Symposia
Table name: Collaboration, Sponsoring, (C)R&D Projects...
Table name: Products Competitors


In [8]:
# Get data from Granta ref papers table
table = 'Granta Reference Papers'
attribute = "Year of Publication"

tableRef = gdl.PartialTableReference(tableName=table)
attrRef = gdl.AttributeReference(name=attribute, DBKey=dbKey, partialTableReference=tableRef)
searchCriterion = gdl.RecordSearchCriterion(searchAttribute=attrRef, existsSearchValue=gdl.ExistsSearchValue())
request = gdl.CriteriaSearch(DBKey=dbKey, searchCriteria=[searchCriterion])

searchResults = session.searchService.CriteriaSearch(request).searchResults

df2 = pd.DataFrame({'ShortName': [r.shortName for r in searchResults],
                     'LongName': [r.longName  for r in searchResults]})
df2

,ShortName,LongName
0,2002_A. Martin-Meizoso_ECF14_Contr,"2002_A. Martin-Meizoso_ECF14, Cracow 2002_Cont..."
1,2002_E.M.A. Maine_Mater_An in,2002_E.M.A. Maine_Materials and Design_An inve...
2,2002_K.W. Johnson_Mater_Group,2002_K.W. Johnson_Materials and Design_Groupin...
3,2002_P. Goodhew_Mater_Cornu,2002_P. Goodhew_Material World_Cornucopia of d...
4,2002_M. Zupan_Advan_Actua,2002_M. Zupan_Advanced Engineering_Actuator Cl...
...,...,...
869,2022_Delaney_ProductDesign,2022_Delaney_ProductDesign
870,2022_Skolek_TheMicrostructureOfCastSteel.pdf,2022_Skolek_TheMicrostructureOfCastSteel.pdf
871,2022_Karothu_DynamicOrganicCrystal,2022_Karothu_DynamicOrganicCrystal
872,2022_Kandemir_DiscocervCervicalDisc,2022_Kandemir_DiscocervCervicalDisc
